This file compiles state-level data.

In [ ]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
import geopandas as geopd
import us
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
# File path
root = ''
result_path = root + 'final_data/'
path_geodata = root + 'Data/geodata/'
path_data = root + 'Data/'

# Vehicle stock

In [ ]:
# Read in AFDC EV statistics
df_cars = pd.read_csv(path_data + 'AFDC/AFDC_allvehicles_2023.csv')
cars_cols = df_cars.columns.tolist()
for col in cars_cols:
    if '_2023' in col:
        df_cars.rename(columns={col:col[:-5]},inplace=True)
    if 'Unnamed' in col:
        df_cars.drop(columns=[col],inplace=True)

In [ ]:
# Rename column
df_cars.set_index('State', inplace=True)
df_cars.rename(columns={'Electric (EV)':'EVs_byState'},inplace=True)
df_cars.rename(columns={'Plug-In Hybrid Electric (PHEV)':'PHEVs_byState'},inplace=True)
df_cars.head(3)

In [ ]:
# Convert to numeric
for col in df_cars.columns:
    try:
        df_cars[col] = df_cars[col].str.replace(',','')
        df_cars[col] = pd.to_numeric(df_cars[col], errors='coerce')
    except:
        print(col)

In [ ]:
# Sum over all fuel types
df_cars['cars_byState'] = df_cars.sum(axis=1)

In [ ]:
# Include STATEFP
for ind in df_cars.index:
    if ind == 'District of Columbia':
        df_cars.loc[ind,'STATEFP'] = '11'
    else:
        df_cars.loc[ind,'STATEFP'] = us.states.lookup(ind).fips

In [ ]:
# Get only relevant columns
df_cars_sel = df_cars[['STATEFP','EVs_byState','PHEVs_byState','cars_byState']].copy()
df_cars_sel.set_index('STATEFP', inplace=True)

# Save

In [ ]:
# Save
df_cars_sel.to_csv(result_path + 'level_state.csv')